In [67]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('C:/Users/Acer/Desktop/ml-homework/Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Acer/Desktop/ml-homework/Data/data_description.txt
C:/Users/Acer/Desktop/ml-homework/Data/sample_submission.csv
C:/Users/Acer/Desktop/ml-homework/Data/test.csv
C:/Users/Acer/Desktop/ml-homework/Data/train.csv


In [68]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [69]:
df = pd.read_csv('C:/Users/Acer/Desktop/ml-homework/Data/train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,

# Deal with Na-s

In [79]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
class DropHighNA(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.5):
        self.threshold = threshold
        self.columns_to_drop_ = []

    def fit(self, X, y=None):
        na_ratios = X.isna().mean()
        self.columns_to_drop_ = na_ratios[na_ratios > self.threshold].index.tolist()
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop_, errors='ignore')
    

class FillInNA(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.num_imputer = SimpleImputer(strategy='mean')
        self.cat_imputer = SimpleImputer(strategy='most_frequent')

    def fit(self, X, y=None):
        self.numeric_cols = X.select_dtypes(exclude='object').columns
        self.categorical_cols = X.select_dtypes(include='object').columns

        self.num_imputer.fit(X[self.numeric_cols])
        self.cat_imputer.fit(X[self.categorical_cols])
        return self

    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.numeric_cols] = self.num_imputer.transform(X_copy[self.numeric_cols])
        X_copy[self.categorical_cols] = self.cat_imputer.transform(X_copy[self.categorical_cols])
        return X_copy

# Make everything numerical

In [71]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
class CustomEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.one_hot_encoder = None
        self.label_encoders = {}
        self.one_hot_cols = []

    def fit(self, X, y=None):
        cat_cols = X.select_dtypes(include=['object']).columns
        self.one_hot_cols = [col for col in cat_cols if X[col].nunique() <= 5]
        high_card_cols = [col for col in cat_cols if X[col].nunique() > 5]

        self.one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
        self.one_hot_encoder.fit(X[self.one_hot_cols])

        for col in high_card_cols:
            le = LabelEncoder()
            le.fit(X[col])
            self.label_encoders[col] = le
        self.feature_names_in_ = X.columns.tolist()
        return self

    def get_feature_names_out(self):
        base_cols = []

        if self.one_hot_encoder:
            base_cols.extend(self.one_hot_encoder.get_feature_names_out(self.one_hot_cols))

        numeric_cols = [
            col for col in self.feature_names_in_
            if col not in self.one_hot_cols and col not in self.label_encoders
        ]
        label_cols = list(self.label_encoders.keys())

        return numeric_cols + label_cols + list(base_cols)

    def transform(self, X):
        X_copy = X.copy()

        one_hot_encoded = self.one_hot_encoder.transform(X_copy[self.one_hot_cols])
        one_hot_df = pd.DataFrame(one_hot_encoded, columns=self.one_hot_encoder.get_feature_names_out(self.one_hot_cols), index=X_copy.index)

        X_copy.drop(columns=self.one_hot_cols, inplace=True)
        X_copy = pd.concat([X_copy, one_hot_df], axis=1)

        for col, le in self.label_encoders.items():
            X_copy[col] = X_copy[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

        return X_copy

# Train

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor

X = df.drop(['Id'], axis=1)
Y = X.pop('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('drop_high_na', DropHighNA()),
    ('impute', FillInNA()),
    ('encode', CustomEncoder()),
    ('scaler', MinMaxScaler()),
])


X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)


model = DecisionTreeRegressor(random_state=42)
model.fit(X_train_transformed, y_train)

train_preds = model.predict(X_train_transformed)
test_preds = model.predict(X_test_transformed)

mse_train = root_mean_squared_error(y_train, train_preds)
mse_test = root_mean_squared_error(y_test, test_preds)

r2_train = r2_score(y_train, train_preds)
r2_test = r2_score(y_test, test_preds)

print("Root Mean Squared Error (Train):", mse_train)
print("Root Mean Squared Error (Test):", mse_test)
print("R2 Score (Train):", r2_train)
print("R2 Score (Test):", r2_test)


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Root Mean Squared Error (Train): 0.0
Root Mean Squared Error (Test): 40468.7717526043
R2 Score (Train): 1.0
R2 Score (Test): 0.7864860707449662


In [63]:
X_train_transformed

array([[-0.8667643 , -0.21289571, -0.03986769, ...,  0.19550504,
        -0.14789275,  0.30263639],
       [ 0.07410996, -0.26524463, -0.70987429, ...,  0.19550504,
        -0.14789275,  0.30263639],
       [-0.63154574, -0.17784146, -0.87737594, ...,  0.19550504,
         6.7616566 , -3.30429529],
       ...,
       [-0.8667643 , -0.23409563, -0.87737594, ...,  0.19550504,
        -0.14789275, -3.30429529],
       [-0.16110861, -0.28337613, -1.04487759, ...,  0.19550504,
        -0.14789275,  0.30263639],
       [ 1.48542135, -0.65139925, -2.0498875 , ...,  0.19550504,
        -0.14789275,  0.30263639]], shape=(1168, 112))

In [90]:
import dagshub
dagshub.init(repo_owner='gbend22', repo_name='ml-homework', mlflow=True)

import mlflow
mlflow.set_experiment("Decision_Tree")

with mlflow.start_run():
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train_transformed, y_train)
    
    pred1 = model.predict(X_train_transformed)
    mse1 = root_mean_squared_error(y_train, pred1)

    pred2 = model.predict(X_test_transformed)
    mse2 = root_mean_squared_error(y_test, pred2)

    # Log metrics and parameters
    mlflow.log_param("Model_Type", "LinearRegression")
    mlflow.log_metric("Root_Mean_Squared_Error_On_Train", mse1)
    mlflow.log_metric("Root_Mean_Squared_Error_On_Test", mse2)

    r1 = r2_score(y_train, pred1)
    r2 = r2_score(y_test, pred2)

    mlflow.log_metric("r2_Score_On_Train", r1*100)
    mlflow.log_metric("r2_Score_On_Test", r2*100)

    mlflow.sklearn.log_model(model, "linear_regression_model")
    mlflow.sklearn.log_model(pipeline, "pipeline")

    mlflow.log_param("test_Size", 0.2)

Initialized MLflow to track repo "gbend22/ml-homework"

Repository gbend22/ml-homework initialized!

2025/04/08 21:02:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/08 21:02:25 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/04/08 21:02:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run judicious-snipe-615 at: https://dagshub.com/gbend22/ml-homework.mlflow/#/experiments/2/runs/bd0da60fb3134b238a000d41539d8b0a
🧪 View experiment at: https://dagshub.com/gbend22/ml-homework.mlflow/#/experiments/2
